In [4]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [6]:
def increment_by_one(number):
    return number + 1

In [7]:
from langchain_core.runnables import RunnableLambda

r = RunnableLambda(increment_by_one)
r.invoke(1)


2

In [8]:
from langchain_core.runnables import RunnableLambda

r = RunnableLambda(lambda x: x+1)
r.invoke(5)


6

In [11]:
from langchain_core.runnables import RunnableLambda, RunnableSequence

sequence = RunnableLambda(lambda x: x + 1) | RunnableLambda(lambda x: x * 2)
sequence.invoke(2)


6

In [14]:
from langchain_core.globals import set_debug

set_debug(False)

In [ ]:
from langchain_core.runnables import RunnableParallel

rp = RunnableParallel(
    {
        "mul_2": RunnableLambda(lambda x: x * 2),
        "mul_5": RunnableLambda(lambda x: x * 5)
    }
)

rp.invoke(2)

[chain/start] [chain:RunnableParallel<mul_2,mul_5>] Entering Chain run with input:
{
  "input": 2
}
[chain/start] [chain:RunnableParallel<mul_2,mul_5> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": 2
}
[chain/end] [chain:RunnableParallel<mul_2,mul_5> > chain:RunnableLambda] s] Exiting Chain run with output:
{
  "output": 3
}
[chain/start] [chain:RunnableParallel<mul_2,mul_5> > chain:RunnableLambda] Entering Chain run with input:
{
  "input": 2
}
[chain/end] [chain:RunnableParallel<mul_2,mul_5> > chain:RunnableLambda] s] Exiting Chain run with output:
{
  "output": 4
}
[chain/end] [chain:RunnableParallel<mul_2,mul_5>] s] Exiting Chain run with output:
{
  "mul_2": 3,
  "mul_5": 4
}


{'mul_2': 3, 'mul_5': 4}

In [16]:
sequence = RunnableLambda(lambda x: x + 1) | {
        "mul_2": RunnableLambda(lambda x: x + 2),
        "mul_5": RunnableLambda(lambda x: x * 5)
    }

sequence.invoke(1)

{'mul_2': 4, 'mul_5': 10}

In [17]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel

review = f"""
Needed a nice lamp for my bedroom, and this one had
additional storage and not too high of a price point.
Got it fast. The string to our lamp broke during the
transit and the company happily sent over a new one.
Came within a few days as well. It was easy to put
together. I had a missing part, so I contacted their
support and they very quickly got me the missing piece!
Lumina seems to me to be a great company that cares
about their customers and products!!"""

summary_prompt = "summarize the below text into one sentence: {text}"
sentiment_prompt = "What is the sentiment of text below: {text}"

summary_prompt_template = PromptTemplate(template = summary_prompt)
sentiment_prompt_template = PromptTemplate(template =  sentiment_prompt)

llm = ChatOpenAI(model = "gpt-4o-mini")

summary_chain = summary_prompt_template | llm | StrOutputParser()
sentiment_chain = sentiment_prompt_template | llm | StrOutputParser()

sequence = RunnableParallel({
    "summary": summary_chain,
    "sentiment": sentiment_chain
})

sequence.invoke({
    "text": review
})

{'summary': "I found a well-priced lamp with extra storage for my bedroom, and despite a minor issue during transit, the company's responsive customer support quickly resolved it, indicating that Lumina is a caring and reliable brand.",
 'sentiment': 'The sentiment of the text is very positive. The reviewer expresses satisfaction with the lamp\'s features, the swift delivery, effective customer service regarding issues, and an overall impression that the company values its customers. Phrases like "happily sent over a new one," "came within a few days," and "great company that cares about their customers" reflect a positive experience.'}

In [18]:
from langchain_core.runnables import RunnablePassthrough
 
rp = RunnablePassthrough()
rp.invoke({
    "k1": "v1",
    "k2": "v2"
}) 


{'k1': 'v1', 'k2': 'v2'}

In [19]:
from langchain_core.runnables import RunnablePassthrough

rp = RunnableParallel({
    "original": RunnablePassthrough(),
    "modified": lambda x: x["num"] + 1
})

rp.invoke({"num": 3})

{'original': {'num': 3}, 'modified': 4}

In [22]:
from langchain_core.runnables import RunnablePick

rp = RunnablePick("k1")
rp.invoke({
    "k1": "v1",
    "k2": "v2"
}) 

'v1'

In [1]:
from langchain_core.runnables import RunnableAssign

mapper = {
    "add_ten": lambda x: x["input"] + 10    
}

ra = RunnableAssign(mapper)
ra.invoke({"input": 5})

{'input': 5, 'add_ten': 15}

In [3]:
from langchain_core.runnables import RunnablePassthrough, RunnableAssign

mapper = {
    "d": RunnablePassthrough()
}

ra = RunnableAssign(mapper)
ra.invoke({"k1": "v1", "k2": "v2"})

{'k1': 'v1', 'k2': 'v2', 'd': {'k1': 'v1', 'k2': 'v2'}}